In [2]:
%pip install simpy

In [4]:
# Resources con prioridad

# En la guardia de un hospital, hay un solo médico de guardia que atiende pacientes de a uno por vez.
# Tres pacientes llegan al mismo tiempo, pero con diferentes niveles de urgencia:

# El paciente 1 tiene un dolor leve  (mínima prioridad)
# El paciente 2 presenta síntomas preocupantes (máxima prioridad)
# El paciente 3 tiene fiebre alta (prioridad media)

# El médico atiende a los pacientes en orden de prioridad médica
# Cada consulta toma 3 minutos exactos.

# Simular la atención médica de los 3 pacientes y mostrar el orden en que
# se atienden y los tiempos de espera.

import simpy

def tarea(env, nombre, recurso, prioridad):
    print(f'{nombre} pide acceso con prioridad {prioridad} en t={env.now}')
    with recurso.request(priority=prioridad) as req:
        yield req
        print(f'{nombre} empieza a usar el recurso en t={env.now}')
        yield env.timeout(3)
        print(f'{nombre} termina en t={env.now}')

env = simpy.Environment()
recurso = simpy.PriorityResource(env, capacity=1)

env.process(tarea(env, 'A', recurso, prioridad=5))
env.process(tarea(env, 'B', recurso, prioridad=2))
env.process(tarea(env, 'C', recurso, prioridad=3))

env.run()

A pide acceso con prioridad 5 en t=0
B pide acceso con prioridad 2 en t=0
C pide acceso con prioridad 3 en t=0
A empieza a usar el recurso en t=0
A termina en t=3
B empieza a usar el recurso en t=3
B termina en t=6
C empieza a usar el recurso en t=6
C termina en t=9


In [6]:
# Stores con Prioridad

# En un centro de distribución, los paquetes llegan al sistema con distintos niveles de urgencia.
# Un operador se encarga de despacharlos de a uno por vez, priorizando los más urgentes.

# Al minuto de iniciada la jornada, se recibe un paquete normal.
# A los 2 minutos, llega un paquete urgente.

# El operador comienza a despachar paquetes a partir del minuto 3 y tarda 2 minutos en procesar cada uno.
# El sistema debe garantizar que el paquete urgente se despache antes que el normal, a pesar de haber llegado después.

# Simular el proceso y mostrar el orden en que los paquetes son despachados.

import simpy

def productor(env, store):
    yield env.timeout(1)
    yield store.put((2, 'normal'))
    print('Agregado normal')
    yield env.timeout(1)
    yield store.put((1, 'urgente'))
    print('Agregado urgente')

def consumidor(env, store):
    yield env.timeout(3)
    while True:
        prioridad, valor = yield store.get()
        print(f'Consumido: {valor} con prioridad {prioridad} en t={env.now}')
        yield env.timeout(2)

env = simpy.Environment()
store = simpy.PriorityStore(env)
env.process(productor(env, store))
env.process(consumidor(env, store))
env.run()

Agregado normal
Agregado urgente
Consumido: urgente con prioridad 1 en t=3
Consumido: normal con prioridad 2 en t=5
